### Dump real data for training to this folder, so that we can train the NIQE model

In [1]:
import os
import math
from abc import abstractmethod
import random
import sys
import uuid
import copy

from PIL import Image
import requests
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from tqdm import tqdm
import matplotlib.pyplot as plt
import h5py
import gc
import copy
import timeit
from tqdm import trange


img_size = 64
root_path = r"D:\LocalWD\CcGAN_TPAMI_NIQE\Cell-200\NIQE_filter_64x64"
h5_file_path = r"D:\BaiduSyncdisk\Baidu_WD\datasets\CCGM_or_regression\Cell200"

seed=2024
random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
cudnn.benchmark = False
np.random.seed(seed)


In [8]:
data_filename = h5_file_path + '/Cell200_{}x{}.h5'.format(img_size, img_size)
hf = h5py.File(data_filename, 'r')
labels = hf['CellCounts'][:]
labels = labels.astype(float)
images = hf['IMGs_grey'][:]
hf.close()
n = len(labels)

In [9]:
min_label=1
max_label=200
num_imgs_per_label=10
label_stepsize=2

# subset
selected_labels = np.arange(min_label, max_label+1)
for i in range(len(selected_labels)):
    curr_label = selected_labels[i]
    index_curr_label = np.where(labels==curr_label)[0]
    if i == 0:
        images_subset = images[index_curr_label]
        labels_subset = labels[index_curr_label]
    else:
        images_subset = np.concatenate((images_subset, images[index_curr_label]), axis=0)
        labels_subset = np.concatenate((labels_subset, labels[index_curr_label]))
# for i
images = images_subset
labels = labels_subset

# for each label select num_imgs_per_label
selected_labels = np.arange(min_label, max_label+1, label_stepsize)
n_unique_labels = len(selected_labels)

for i in range(n_unique_labels):
    curr_label = selected_labels[i]
    index_curr_label = np.where(labels==curr_label)[0]
    if i == 0:
        images_subset = images[index_curr_label[0:num_imgs_per_label]]
        labels_subset = labels[index_curr_label[0:num_imgs_per_label]]
    else:
        images_subset = np.concatenate((images_subset, images[index_curr_label[0:num_imgs_per_label]]), axis=0)
        labels_subset = np.concatenate((labels_subset, labels[index_curr_label[0:num_imgs_per_label]]))
# for i
images = images_subset
labels = labels_subset

In [11]:
dump_fake_images_folder = root_path + '/real_data/Cell200_images_0_10_200/images'
os.makedirs(dump_fake_images_folder, exist_ok=True)

for i in trange(len(labels)):
    label_i = labels[i]
    filename_i = dump_fake_images_folder + "/{}_{}.png".format(i, label_i)
    os.makedirs(os.path.dirname(filename_i), exist_ok=True)
    image_i = images[i][0]
    image_i_pil = Image.fromarray(image_i)
    image_i_pil.save(filename_i)
#end for i

100%|██████████| 1000/1000 [00:00<00:00, 1730.14it/s]
